In [ ]:
import boto3
import random

s3client = boto3.client('s3')

In [ ]:
# Bucket names are globally unique. Here we'll add a random element to the name to increase the odds of
# avoiding a name collision
bucket1 = 'ds-e1-' + str(random.randint(1, 9999999))
print bucket1

bucket2 = 'ds-e2-' + str(random.randint(1, 9999999))
print bucket2

In [ ]:
# Create bucket 1
response = s3client.create_bucket(
    Bucket=bucket1,
    CreateBucketConfiguration={
        'LocationConstraint': 'us-west-1'
    }
)

print response

In [ ]:
# Create bucket 2
response = s3client.create_bucket(
    Bucket=bucket2,
    CreateBucketConfiguration={
        'LocationConstraint': 'eu-west-1'
    }
)

print response

In [ ]:
# Versioning needs to be enable for cross region replication
response = s3client.put_bucket_versioning(
    Bucket=bucket1,
    VersioningConfiguration={
        'MFADelete': 'Disabled',
        'Status': 'Enabled'
    }
)

print response

response = s3client.put_bucket_versioning(
    Bucket=bucket2,
    VersioningConfiguration={
        'MFADelete': 'Disabled',
        'Status': 'Enabled'
    }
)

print response

In [ ]:
assume_role_policy_doc = """{
   "Version":"2012-10-17",
   "Statement":[
      {
         "Effect":"Allow",
         "Principal":{
            "Service":"s3.amazonaws.com"
         },
         "Action":"sts:AssumeRole"
      }
   ]
}"""

In [ ]:
iam_client = boto3.client('iam')
response = iam_client.create_role(
    AssumeRolePolicyDocument=assume_role_policy_doc,
    Path='/',
    RoleName='My-S3-Rep-Role',
)

print response

roleArn = response['Role']['Arn']
print roleArn

In [ ]:
permissions_policy = """{
   "Version":"2012-10-17",
   "Statement":[
      {
         "Effect":"Allow",
         "Action":[
            "s3:GetObjectVersion",
            "s3:GetObjectVersionAcl"
         ],
         "Resource":[
            "arn:aws:s3:::""" + bucket1 + """/*"
         ]
      },
      {
         "Effect":"Allow",
         "Action":[
            "s3:ListBucket",
            "s3:GetReplicationConfiguration"
         ],
         "Resource":[
            "arn:aws:s3:::""" + bucket1 + """"
         ]
      },
      {
         "Effect":"Allow",
         "Action":[
            "s3:ReplicateObject",
            "s3:ReplicateDelete"
         ],
         "Resource":"arn:aws:s3:::""" + bucket2 + """/*"
      }
   ]
}"""



In [ ]:
response = iam_client.create_policy(
    PolicyName='My-S3-Repl-Policy',
    Path='/',
    PolicyDocument=permissions_policy
)

print response

policyArn = response['Policy']['Arn']
print policyArn

In [ ]:
# Attach policy to role
response = iam_client.attach_role_policy(
    RoleName='My-S3-Rep-Role',
    PolicyArn=policyArn
)

print response

In [ ]:
# Enable replication
response = s3client.put_bucket_replication(
    Bucket=bucket1,
    ReplicationConfiguration={
        'Role': roleArn,
        'Rules': [
            {
                'ID': 'string',
                'Prefix': '',
                'Status': 'Enabled',
                'Destination': {
                    'Bucket': 'arn:aws:s3:::' + bucket2,
                    'StorageClass': 'STANDARD'
                }
            },
        ]
    }
)

print response

In [ ]:
f = open("./yo.txt","w")
f.write("yo, world")
f.close()

In [ ]:
#Upload to bucket 1
s3 = boto3.resource('s3')
response = s3.meta.client.upload_file('./yo.txt', bucket1, 'yo.txt')
print response

In [ ]:
# List objects in bucket2
response = s3client.list_objects_v2(
    Bucket=bucket2
)

print response

In [ ]:
# Detach policy
response = iam_client.detach_role_policy(
    RoleName='My-S3-Rep-Role',
    PolicyArn=policyArn
)

print response

In [ ]:
# Delete policy
response = iam_client.delete_policy(
    PolicyArn=policyArn
)

print response

In [ ]:
# Delete role
response = iam_client.delete_role(
    RoleName='My-S3-Rep-Role',
)

print response

In [ ]:
# Clean up - bucket 1
response = s3client.list_object_versions(
    Bucket=bucket1,
    Prefix='',
    MaxKeys=100
)


print response

if 'Versions' in response:
    versions = response['Versions']
    for v in versions:
        print v['Key']
else:
    print 'No versions'
    


for version in versions:
    response = s3client.delete_object(
        Bucket=bucket1,
        Key=version['Key'],
        VersionId=version['VersionId']
    )

    print response

response = s3client.delete_bucket(
    Bucket=bucket1
)

print response

In [ ]:
response = s3client.list_object_versions(
    Bucket=bucket2,
    Prefix='',
    MaxKeys=100
)


print response

if 'Versions' in response:
    versions = response['Versions']
    for v in versions:
        print v['Key']
else:
    print 'No versions'
    


for version in versions:
    response = s3client.delete_object(
        Bucket=bucket2,
        Key=version['Key'],
        VersionId=version['VersionId']
    )

    print response

response = s3client.delete_bucket(
    Bucket=bucket2
)

print response